In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r /content/gdrive/MyDrive/cripp_trial /content

In [ ]:
cd /content/cripp_trial

/content/cripp_trial


In [ ]:
!pip install PyYAML==5.4

In [ ]:
import os
import argparse
import json
import time
import numpy as np
import torch
torch.backends.cudnn.enabled  = True
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from configs.config import cfg, merge_cfg_from_file
#from datasets.datasets import create_dataset
from models.modules import ChangeDetectorDoubleAttDyn, AddSpatialInfo
from models.dynamic_speaker import DynamicSpeaker

from utils.utils import AverageMeter, accuracy, set_mode, load_checkpoint, \
                        decode_sequence, coco_gen_format_save
from utils.vis_utils import visualize_att
from tqdm import tqdm

In [ ]:
device = "cude" if torch.cuda.is_available() else 'cpu'

In [ ]:
cfg = '/content/cripp_trial/configs/dynamic/dynamic.yaml'
visualize = False
snapshot = '/content/cripp_trial/experiments/snapshots/dynamic_checkpoint_9000.pt'
gpu=-1

In [ ]:
'''
conifgure the cfg file with correct data paths


__C.exp_dir = '/content/cripp_trial/inference'
__C.exp_name = 'dynamic'

__C.data.dataset = 'rcc_dataset'
__C.data.num_workers = 8
__C.data.default_feature_dir = '/content/cripp_trial/features'
__C.data.semantic_feature_dir = '/content/cripp_trial/data/sc_features'
__C.data.nonsemantic_feature_dir = '/content/cripp_trial/data/nsc_features'
__C.data.default_img_dir = '/content/cripp_trial/images'
__C.data.semantic_img_dir = '/content/cripp_trial/data/sc_images'
__C.data.nonsemantic_img_dir = '/content/cripp_trial/data/nsc_images'
__C.data.vocab_json = '/content/cripp_trial/jsons/vocab.json'
__C.data.splits_json = '/content/cripp_trial/jsons/splits.json'
__C.data.h5_label_file = '/content/cripp_trial/jsons/labels.h5'
__C.data.h5_ref_label_file = '/content/cripp_trial/jsons/labels.h5'
__C.data.type_mapping_json = '/content/cripp_trial/jsons/type_mapping.json'

'''

"\nconifgure the cfg file with correct data paths\n\n\n__C.exp_dir = '/content/cripp_trial/inference'\n__C.exp_name = 'dynamic'\n\n__C.data.dataset = 'rcc_dataset'\n__C.data.num_workers = 8\n__C.data.default_feature_dir = '/content/cripp_trial/features'\n__C.data.semantic_feature_dir = '/content/cripp_trial/data/sc_features'\n__C.data.nonsemantic_feature_dir = '/content/cripp_trial/data/nsc_features'\n__C.data.default_img_dir = '/content/cripp_trial/images'\n__C.data.semantic_img_dir = '/content/cripp_trial/data/sc_images'\n__C.data.nonsemantic_img_dir = '/content/cripp_trial/data/nsc_images'\n__C.data.vocab_json = '/content/cripp_trial/jsons/vocab.json'\n__C.data.splits_json = '/content/cripp_trial/jsons/splits.json'\n__C.data.h5_label_file = '/content/cripp_trial/jsons/labels.h5'\n__C.data.h5_ref_label_file = '/content/cripp_trial/jsons/labels.h5'\n__C.data.type_mapping_json = '/content/cripp_trial/jsons/type_mapping.json'\n\n"

In [ ]:
'''
in dynamic.yml changes

input_dim: 2048 + 4
input_dim: 1024 + 2


vocab_json: '/content/cripp_trial/jsons/vocab.json'
    h5_label_file: '/content/cripp_trial/jsons/labels.h5'
    type_mapping_json: '/content/cripp_trial/jsons/type_mapping.json'


'''

"\nin dynamic.yml changes\n\ninput_dim: 2048 + 4\ninput_dim: 1024 + 2\n\n\nvocab_json: '/content/cripp_trial/jsons/vocab.json'\n    h5_label_file: '/content/cripp_trial/jsons/labels.h5'\n    type_mapping_json: '/content/cripp_trial/jsons/type_mapping.json'\n\n\n"

In [ ]:
merge_cfg_from_file(cfg)

In [ ]:
#create an inference directory
exp_dir = '/content/cripp_trial/inference2'
exp_name = 'dynamic'
output_dir = os.path.join(exp_dir, exp_name)
test_output_dir = os.path.join(output_dir, 'test_output')
if not os.path.exists(test_output_dir):
    os.makedirs(test_output_dir)
caption_output_path = os.path.join(test_output_dir, 'captions', 'test')
if not os.path.exists(caption_output_path):
    os.makedirs(caption_output_path)
att_output_path = os.path.join(test_output_dir, 'attentions', 'test')
if not os.path.exists(att_output_path):
    os.makedirs(att_output_path)

In [ ]:
#snapshot_dir = os.path.join(output_dir, 'snapshots')
#snapshot_file = '%s_checkpoint_%d.pt' % (exp_name, snapshot)
snapshot_full_path = '/content/cripp_trial/experiments/snapshots/dynamic_checkpoint_9000.pt'
checkpoint = load_checkpoint(snapshot_full_path)
change_detector_state = checkpoint['change_detector_state']
speaker_state = checkpoint['speaker_state']

Loading checkpoint from /content/cripp_trial/experiments/snapshots/dynamic_checkpoint_9000.pt


##Modules.py File

In [ ]:
import json
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F




class ChangeDetectorDoubleAttDyn(nn.Module):

    def __init__(self):
        super().__init__()
        self.input_dim = 2052
        self.dim = 128

        self.embed = nn.Sequential(
            nn.Conv2d(self.input_dim, self.dim, kernel_size=1, padding=0),
            nn.GroupNorm(32, self.dim),
            nn.Dropout(0.5),
            nn.ReLU()
        )

        self.att = nn.Conv2d(self.dim, 1, kernel_size=1, padding=0)
        self.fc1 = nn.Linear(self.input_dim // 2, 6)

    def forward(self, input_1, input_2):
        batch_size, _, H, W = input_1.size()
        input_diff = input_2 - input_1
        input_before = torch.cat([input_1, input_diff], 1)
        input_after = torch.cat([input_2, input_diff], 1)
        embed_before = self.embed(input_before)
        embed_after = self.embed(input_after)
        att_weight_before = F.sigmoid(self.att(embed_before))
        att_weight_after = F.sigmoid(self.att(embed_after))

        att_1_expand = att_weight_before.expand_as(input_1)
        attended_1 = (input_1 * att_1_expand).sum(2).sum(2)  # (batch, dim)
        att_2_expand = att_weight_after.expand_as(input_2)
        attended_2 = (input_2 * att_2_expand).sum(2).sum(2)  # (batch, dim)
        input_attended = attended_2 - attended_1
        pred = self.fc1(input_attended)

        return pred, att_weight_before, att_weight_after, attended_1, attended_2, input_attended


class AddSpatialInfo(nn.Module):

    def _create_coord(self, img_feat):
        batch_size, _, h, w = img_feat.size()
        coord_map = img_feat.new_zeros(2, h, w)
        for i in range(h):
            for j in range(w):
                coord_map[0][i][j] = (j * 2.0 / w) - 1
                coord_map[1][i][j] = (i * 2.0 / h) - 1
        sequence = [coord_map] * batch_size
        coord_map_in_batch = torch.stack(sequence)
        return coord_map_in_batch

    def forward(self, img_feat):
        coord_map = self._create_coord(img_feat)
        img_feat_aug = torch.cat([img_feat, coord_map], dim=1)
        return img_feat_aug


##CaptionModel.py File

In [ ]:
# This file contains ShowAttendTell and AllImg model

# ShowAttendTell is from Show, Attend and Tell: Neural Image Caption Generation with Visual Attention
# https://arxiv.org/abs/1502.03044

# AllImg is a model where
# img feature is concatenated with word embedding at every time step as the input of lstm
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from functools import reduce

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import *


class CaptionModel(nn.Module):
    def __init__(self):
        super(CaptionModel, self).__init__()

    # implements beam search
    # calls beam_step and returns the final set of beams
    # augments log-probabilities with diversity terms when number of groups > 1

    def forward(self, *args, **kwargs):
        mode = kwargs.get('mode', 'forward')
        if 'mode' in kwargs:
            del kwargs['mode']
        return getattr(self, '_'+mode)(*args, **kwargs)

    def beam_search(self, init_state, init_logprobs, *args, **kwargs):

        # function computes the similarity score to be augmented
        def add_diversity(beam_seq_table, logprobsf, t, divm, diversity_lambda, bdash):
            local_time = t - divm
            unaug_logprobsf = logprobsf.clone()
            for prev_choice in range(divm):
                prev_decisions = beam_seq_table[prev_choice][local_time]
                for sub_beam in range(bdash):
                    for prev_labels in range(bdash):
                        logprobsf[sub_beam][prev_decisions[prev_labels]] = logprobsf[sub_beam][prev_decisions[prev_labels]] - diversity_lambda
            return unaug_logprobsf

        # does one step of classical beam search

        def beam_step(logprobsf, unaug_logprobsf, beam_size, t, beam_seq, beam_seq_logprobs, beam_logprobs_sum, state):
            #INPUTS:
            #logprobsf: probabilities augmented after diversity
            #beam_size: obvious
            #t        : time instant
            #beam_seq : tensor contanining the beams
            #beam_seq_logprobs: tensor contanining the beam logprobs
            #beam_logprobs_sum: tensor contanining joint logprobs
            #OUPUTS:
            #beam_seq : tensor containing the word indices of the decoded captions
            #beam_seq_logprobs : log-probability of each decision made, same size as beam_seq
            #beam_logprobs_sum : joint log-probability of each beam

            ys,ix = torch.sort(logprobsf,1,True)
            candidates = []
            cols = min(beam_size, ys.size(1))
            rows = beam_size
            if t == 0:
                rows = 1
            for c in range(cols): # for each column (word, essentially)
                for q in range(rows): # for each beam expansion
                    #compute logprob of expanding beam q with word in (sorted) position c
                    local_logprob = ys[q,c].item()
                    candidate_logprob = beam_logprobs_sum[q] + local_logprob
                    local_unaug_logprob = unaug_logprobsf[q,ix[q,c]]
                    candidates.append({'c':ix[q,c], 'q':q, 'p':candidate_logprob, 'r':local_unaug_logprob})
            candidates = sorted(candidates,  key=lambda x: -x['p'])
            
            new_state = [_.clone() for _ in state]
            #beam_seq_prev, beam_seq_logprobs_prev
            if t >= 1:
            #we''ll need these as reference when we fork beams around
                beam_seq_prev = beam_seq[:t].clone()
                beam_seq_logprobs_prev = beam_seq_logprobs[:t].clone()
            for vix in range(beam_size):
                v = candidates[vix]
                #fork beam index q into index vix
                if t >= 1:
                    beam_seq[:t, vix] = beam_seq_prev[:, v['q']]
                    beam_seq_logprobs[:t, vix] = beam_seq_logprobs_prev[:, v['q']]
                #rearrange recurrent states
                for state_ix in range(len(new_state)):
                #  copy over state in previous beam q to new beam at vix
                    new_state[state_ix][:, vix] = state[state_ix][:, v['q']] # dimension one is time step
                #append new end terminal at the end of this beam
                beam_seq[t, vix] = v['c'] # c'th word is the continuation
                beam_seq_logprobs[t, vix] = v['r'] # the raw logprob here
                beam_logprobs_sum[vix] = v['p'] # the new (sum) logprob along this beam
            state = new_state
            return beam_seq,beam_seq_logprobs,beam_logprobs_sum,state,candidates

        # Start diverse_beam_search
        beam_size = 10
        group_size = 1
        diversity_lambda = 0.5
        decoding_constraint = 0
        max_ppl = 0
        bdash = beam_size // group_size # beam per group

        # INITIALIZATIONS
        beam_seq_table = [torch.LongTensor(self.seq_length, bdash).zero_() for _ in range(group_size)]
        beam_seq_logprobs_table = [torch.FloatTensor(self.seq_length, bdash).zero_() for _ in range(group_size)]
        beam_logprobs_sum_table = [torch.zeros(bdash) for _ in range(group_size)]

        # logprobs # logprobs predicted in last time step, shape (beam_size, vocab_size)
        done_beams_table = [[] for _ in range(group_size)]
        state_table = [list(torch.unbind(_)) for _ in torch.stack(init_state).chunk(group_size, 2)]
        logprobs_table = list(init_logprobs.chunk(group_size, 0))
        # END INIT

        # Chunk elements in the args
        args = list(args)
        args = [_.chunk(group_size) if _ is not None else [None]*group_size for _ in args]
        args = [[args[i][j] for i in range(len(args))] for j in range(group_size)]

        for t in range(self.seq_length + group_size - 1):
            for divm in range(group_size): 
                if t >= divm and t <= self.seq_length + divm - 1:
                    # add diversity
                    logprobsf = logprobs_table[divm].data.float()
                    # suppress previous word
                    if decoding_constraint and t-divm > 0:
                        logprobsf.scatter_(1, beam_seq_table[divm][t-divm-1].unsqueeze(1).to(device), float('-inf'))
                    # suppress UNK tokens in the decoding (here <UNK> has an index of 1)
                    logprobsf[:, 1] = logprobsf[:, 1] - 1000  
                    # diversity is added here
                    # the function directly modifies the logprobsf values and hence, we need to return
                    # the unaugmented ones for sorting the candidates in the end. # for historical
                    # reasons :-)
                    unaug_logprobsf = add_diversity(beam_seq_table,logprobsf,t,divm,diversity_lambda,bdash)

                    # infer new beams
                    beam_seq_table[divm],\
                    beam_seq_logprobs_table[divm],\
                    beam_logprobs_sum_table[divm],\
                    state_table[divm],\
                    candidates_divm = beam_step(logprobsf,
                                                unaug_logprobsf,
                                                bdash,
                                                t-divm,
                                                beam_seq_table[divm],
                                                beam_seq_logprobs_table[divm],
                                                beam_logprobs_sum_table[divm],
                                                state_table[divm])

                    # if time's up... or if end token is reached then copy beams
                    for vix in range(bdash):
                        if beam_seq_table[divm][t-divm,vix] == 0 or t == self.seq_length + divm - 1:
                            final_beam = {
                                'seq': beam_seq_table[divm][:, vix].clone(), 
                                'logps': beam_seq_logprobs_table[divm][:, vix].clone(),
                                'unaug_p': beam_seq_logprobs_table[divm][:, vix].sum().item(),
                                'p': beam_logprobs_sum_table[divm][vix].item()
                            }
                            if max_ppl:
                                final_beam['p'] = final_beam['p'] / (t-divm+1)
                            done_beams_table[divm].append(final_beam)
                            # don't continue beams from finished sequences
                            beam_logprobs_sum_table[divm][vix] = -1000

                    # move the current group one step forward in time
                    
                    it = beam_seq_table[divm][t-divm]
                    logprobs_table[divm], state_table[divm] = self.get_logprobs_state(it.to(device), *(args[divm] + [state_table[divm]]))

        # all beams are sorted by their log-probabilities
        done_beams_table = [sorted(done_beams_table[i], key=lambda x: -x['p'])[:bdash] for i in range(group_size)]
        done_beams = reduce(lambda a,b:a+b, done_beams_table)
        return done_beams


##DynamicSpeaker.py File

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from utils.attr_dict import AttrDict
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import PackedSequence, pack_padded_sequence, pad_packed_sequence

speaker_rnn_size = 512
speaker_input_dim = 1026
speaker_word_embed_size = 300
model_speaker = AttrDict()
drop_prob_lm = 0.5
embed_input_dim = 3078
embed_dim = 512

def sort_pack_padded_sequence(input, lengths):
    sorted_lengths, indices = torch.sort(lengths, descending=True)
    tmp = pack_padded_sequence(input[indices], sorted_lengths, batch_first=True)
    inv_ix = indices.clone()
    inv_ix[indices] = inv_ix.new_tensor(torch.arange(0, len(indices)))
    return tmp, inv_ix


def pad_unsort_packed_sequence(input, inv_ix):
    tmp, _ = pad_packed_sequence(input, batch_first=True)
    tmp = tmp[inv_ix]
    return tmp


def last_timestep(unpacked, lengths):
    # Index of the last output for each sequence.
    idx = (lengths - 1).view(-1, 1).expand(unpacked.size(0),
                                           unpacked.size(2)).unsqueeze(1)
    return unpacked.gather(1, idx).squeeze()


def pack_wrapper(module, att_feats, att_masks):
    if att_masks is not None:
        packed, inv_ix = sort_pack_padded_sequence(att_feats, att_masks.data.long().sum(1))
        return pad_unsort_packed_sequence(PackedSequence(module(packed[0]), packed[1]), inv_ix)
    else:
        return module(att_feats)


class DynamicCore(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn_num_layers = 2
        self.drop_prob_lm = 0.5
        self.input_dim = speaker_input_dim
        self.embed_input_dim = 3078
        self.embed_dim = 512

        self.embed = nn.Sequential(
            nn.Linear(self.embed_input_dim, self.embed_dim),
            nn.ReLU(),
            nn.Dropout(self.drop_prob_lm)
        )

        self.module_att_lstm = nn.LSTMCell(self.embed_dim + speaker_rnn_size,
                                           speaker_rnn_size)

        self.weight_fc = nn.Sequential(
            nn.Linear(speaker_rnn_size, 3),
            nn.Softmax(dim=1)
        )

        self.lang_lstm = nn.LSTMCell(speaker_word_embed_size + \
                                     speaker_input_dim,
                                     speaker_rnn_size)
        self.module_weights = None


    def forward(self, xt,
                loc_feat_bef,loc_feat_aft,
                feat_diff, state):

        prev_h = state[0][-1]  # prev hidden state from the lang_lstm
        embed_input = torch.cat([loc_feat_bef, feat_diff, loc_feat_aft], 1)
        embed = self.embed(embed_input)
        module_att_lstm_input = torch.cat([embed, prev_h], 1)

        h_mod_att, c_mod_att = self.module_att_lstm(module_att_lstm_input, (state[0][0], state[1][0]))
        module_weights = self.weight_fc(h_mod_att)
        self.module_weights = module_weights

        feats = torch.cat([loc_feat_bef.unsqueeze(1),
                           feat_diff.unsqueeze(1),
                           loc_feat_aft.unsqueeze(1)], 1)

        weights_expand = module_weights.unsqueeze(2).expand_as(feats)
        att_feat = (feats * weights_expand).sum(1)  # (batch, feat_dim)

        lang_lstm_input = torch.cat([xt, att_feat], 1)
        h_lang, c_lang = self.lang_lstm(lang_lstm_input, (state[0][1], state[1][1]))

        output = F.dropout(h_lang, self.drop_prob_lm, self.training)
        state = (torch.stack([h_mod_att, h_lang]), torch.stack([c_mod_att, c_lang]))

        return output, state


    def get_module_weights(self):
        # needs to be called after forward call
        return self.module_weights


class DynamicSpeaker(CaptionModel):
    def __init__(self):
        super(DynamicSpeaker, self).__init__()
        self.vocab_size = 76
        self.word_embed_size = speaker_word_embed_size
        self.rnn_size = speaker_rnn_size
        self.drop_prob_lm = 0.5
        self.seq_length = 23

        self.ss_prob = 0.0  # Scheduled sampling probability

        self.embed = nn.Sequential(
            nn.Embedding(self.vocab_size, self.word_embed_size),
            nn.ReLU(),
            nn.Dropout(self.drop_prob_lm))

        self.core = DynamicCore()
        self.rnn_num_layers = self.core.rnn_num_layers

        self.logit_layers = getattr(model_speaker, 'logit_layers', 1)
        if self.logit_layers == 1:
            self.logit = nn.Linear(self.rnn_size, self.vocab_size)
        else:
            self.logit = [[nn.Linear(self.rnn_size, self.rnn_size),
                           nn.ReLU(),
                           nn.Dropout(0.5)] \
                          for _ in range(model_speaker.logit_layers - 1)]
            self.logit = nn.Sequential(*(
                    reduce(lambda x, y: x + y, self.logit) + \
                    [nn.Linear(self.rnn_size, self.vocab_size)]))

        self.module_weights = []


    def init_hidden(self, batch_size):
        weight = next(self.parameters())
        return (weight.new_zeros(self.rnn_num_layers, batch_size, self.rnn_size),
                weight.new_zeros(self.rnn_num_layers, batch_size, self.rnn_size))


    def _forward(self,
                 feat_bef, feat_aft,
                 feat_diff, seq):

        # start fresh
        self.module_weights = []

        batch_size = feat_bef.size(0)
        state = self.init_hidden(batch_size)

        # outputs are logprobs
        outputs = feat_bef.new_zeros(batch_size, self.seq_length, self.vocab_size)

        for i in range(self.seq_length):
            if self.training and i >= 1 and self.ss_prob > 0.0:
                sample_prob = feat_bef.new(batch_size).uniform_(0, 1)
                sample_mask = sample_prob < self.ss_prob
                if sample_mask.sum() == 0:
                    it = seq[:, i].clone()
                else:
                    sample_ind = sample_mask.nonzero().view(-1)  # idx for sampling from batch
                    it = seq[:, i].data.clone()
                    prob_prev = torch.exp(outputs[:, i - 1].detach())  # fetch previous distribution (N x (M+1))
                    it.index_copy_(0, sample_ind,
                                   torch.multinomial(prob_prev, 1).view(-1).index_select(0, sample_ind))
            else:
                it = seq[:, i].clone()
            # break if all the sequences end
            if i >= 1 and seq[:, i].sum() == 0:
                break

            output, state = self.get_logprobs_state(it,
                                                    feat_bef, feat_aft,
                                                    feat_diff, state)
            outputs[:, i] = output

        return outputs


    def get_logprobs_state(self, it,
                           feat_bef, feat_aft,
                           feat_diff, state):

        # 'it' contains word indices
        xt = self.embed(it)

        output, state = self.core(xt,
                                  feat_bef, feat_aft,
                                  feat_diff, state)

        # after every call of the core, collect the module weights
        self.module_weights.append(self.core.get_module_weights())
        log_probs = F.log_softmax(self.logit(output), dim=1)
        return log_probs, state


    def get_module_weights(self):
        if len(self.module_weights) == 0:
            print('no module weights accumulated')
            return None
        module_weights_stacked = torch.stack(self.module_weights, dim=1)
        return module_weights_stacked


    def _sample_beam(self,
                     feat_bef, feat_aft,
                     feat_diff):
        # start fresh
        self.module_weights = []

        beam_size = 10
        batch_size = feat_bef.size(0)

        assert beam_size <= self.vocab_size, 'lets assume this for now, otherwise this corner case causes a few headaches down the road. can be dealt with in future if needed'
        seq = torch.LongTensor(self.seq_length, batch_size).zero_()
        seq_logprobs = torch.FloatTensor(self.seq_length, batch_size)

        self.done_beams = [[] for _ in range(batch_size)]
        for k in range(batch_size):
            state = self.init_hidden(beam_size)
            tmp_feat_bef = feat_bef[k:k + 1].expand(beam_size, -1).contiguous()
            tmp_feat_aft = feat_aft[k:k + 1].expand(beam_size, -1).contiguous()
            tmp_feat_diff = feat_diff[k:k + 1].expand(beam_size, -1).contiguous()

            # input <bos> (idx of 2)
            it = feat_bef.new_zeros([beam_size], dtype=torch.long) + 2
            logprobs, state = self.get_logprobs_state(it,
                                                      tmp_feat_bef, tmp_feat_aft,
                                                      tmp_feat_diff, state)

            self.done_beams[k] = self.beam_search(state, logprobs,
                                                  tmp_feat_bef, tmp_feat_aft,
                                                  tmp_feat_diff, state)
            seq[:, k] = self.done_beams[k][0]['seq']
            seq_logprobs[:, k] = self.done_beams[k][0]['logps']

        # return the samples and their log likelihoods
        return seq.transpose(0, 1), seq_logprobs.transpose(0, 1)


    def _sample(self,
                feat_bef, feat_aft,
                feat_diff, seq, sample_max=0):

        # start fresh
        self.module_weights = []

        #sample_max = cfg.model.speaker.get('sample_max', 1)
        beam_size = 1
        temperature = 1.0
        decoding_constraint = 0

        if beam_size > 1:
            return self._sample_beam(feat_bef, feat_aft,
                                     feat_diff)

        batch_size = feat_bef.size(0)
        state = self.init_hidden(batch_size)

        seq = feat_bef.new_zeros((batch_size, self.seq_length), dtype=torch.long)
        seq_logprobs = feat_bef.new_zeros(batch_size, self.seq_length)

        for t in range(self.seq_length + 1):
            if t == 0:  # input <bos>
                it = feat_bef.new_zeros(batch_size, dtype=torch.long) + 2

            logprobs, state = self.get_logprobs_state(it,
                                                      feat_bef, feat_aft,
                                                      feat_diff, state)

            # if first step, make sure we don't sample NULL
            if t == 0:
                tmp = logprobs.new_zeros(logprobs.size())
                tmp[:, 0] = float('-inf')
                logprobs = logprobs + tmp
            # decoding constraint for not sampling the word sampled at t-1
            if decoding_constraint and t > 0:
                tmp = logprobs.new_zeros(logprobs.size())
                tmp.scatter_(1, seq[:, t - 1].data.unsqueeze(1), float('-inf'))
                logprobs = logprobs + tmp

            # sample the next word
            if t == self.seq_length:  # skip if we achieve maximum length
                break
            if sample_max:
                sample_logprobs, it = torch.max(logprobs.data, 1)
                it = it.view(-1).long()
            else:
                if temperature == 1.0:
                    prob_prev = torch.exp(logprobs.data)
                else:
                    prob_prev = torch.exp(torch.div(logprobs.data, temperature))
                it = torch.multinomial(prob_prev, 1)
                sample_logprobs = logprobs.gather(1, it)  # gather the logprobs at sampled positions
                it = it.view(-1).long()  # and flatten indices for downstream processing

            # stop when all finished
            if t == 0:
                unfinished = it > 0
            else:
                unfinished = unfinished * (it > 0)
            it = it * unfinished.type_as(it)
            seq[:, t] = it
            seq_logprobs[:, t] = sample_logprobs.view(-1)
            # quit loop if all sequences have finished
            if unfinished.sum() == 0:
                break

        return seq, seq_logprobs



##Instanciating the Module classes

In [ ]:
# Load modules
change_detector = ChangeDetectorDoubleAttDyn()
change_detector.load_state_dict(change_detector_state)
change_detector = change_detector.to(device)

speaker = DynamicSpeaker()
speaker.load_state_dict(speaker_state)
speaker.to(device)

spatial_info = AddSpatialInfo()
spatial_info.to(device)

print(change_detector)
print(speaker)
print(spatial_info)

ChangeDetectorDoubleAttDyn(
  (embed): Sequential(
    (0): Conv2d(2052, 128, kernel_size=(1, 1), stride=(1, 1))
    (1): GroupNorm(32, 128, eps=1e-05, affine=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): ReLU()
  )
  (att): Conv2d(128, 1, kernel_size=(1, 1), stride=(1, 1))
  (fc1): Linear(in_features=1026, out_features=6, bias=True)
)
DynamicSpeaker(
  (embed): Sequential(
    (0): Embedding(76, 300)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (core): DynamicCore(
    (embed): Sequential(
      (0): Linear(in_features=3078, out_features=512, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
    )
    (module_att_lstm): LSTMCell(1024, 512)
    (weight_fc): Sequential(
      (0): Linear(in_features=512, out_features=3, bias=True)
      (1): Softmax(dim=1)
    )
    (lang_lstm): LSTMCell(1326, 512)
  )
  (logit): Linear(in_features=512, out_features=76, bias=True)
)
AddSpatialInfo()


##Renaming SC_features and SC_images

In [ ]:
import os
path = '/content/cripp_trial/data/sc_images'
files = os.listdir(path)


for index, file in enumerate(files):
  if (index<10):
    os.rename(os.path.join(path, file), os.path.join(path, ''.join(["img_000", str(index), '.png'])))
  else:
    os.rename(os.path.join(path, file), os.path.join(path, ''.join(["img_00", str(index), '.png'])))

In [ ]:
path = '/content/cripp_trial/data/sc_features'
#path='/content/cripp_trial/data/resnet34/sc_features'
files = os.listdir(path)


for index, file in enumerate(files):
  if (index<10):
    os.rename(os.path.join(path, file), os.path.join(path, ''.join(["img_000", str(index), '.png.npy'])))
  else:
    os.rename(os.path.join(path, file), os.path.join(path, ''.join(["img_00", str(index), '.png.npy'])))

##RCCDataset.py File

In [ ]:
import os
import json
import numpy as np
import random
import time

import h5py
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate

vocab_json='/content/cripp_trial/jsons/vocab.json'
type_mapping_json = '/content/cripp_trial/jsons/type_mapping.json'
default_feature_dir = '/content/cripp_trial/data/features'
semantic_feature_dir = '/content/cripp_trial/data/sc_features'
nonsemantic_feature_dir = '/content/cripp_trial/data/nsc_features'
default_img_dir = '/content/cripp_trial/data/images'
semantic_img_dir = '/content/cripp_trial/data/sc_images'
nonsemantic_img_dir = '/content/cripp_trial/data/nsc_images'
splits_json = '/content/cripp_trial/splits.json'
train_batch_size = 128
max_samples = None
val_batch_size = 64
test_batch_size = 1
h5_label_file = '/content/cripp_trial/jsons/labels.h5'



class RCCDataset(Dataset):

    shapes = set(['ball', 'block', 'cube', 'cylinder', 'sphere'])
    sphere = set(['ball', 'sphere'])
    cube = set(['block', 'cube'])
    cylinder = set(['cylinder'])

    colors = set(['red', 'cyan', 'brown', 'blue', 'purple', 'green', 'gray', 'yellow'])

    materials = set(['metallic', 'matte', 'rubber', 'shiny', 'metal'])
    rubber = set(['matte', 'rubber'])
    metal = set(['metal', 'metallic', 'shiny'])

    type_to_label = {
        'color': 0,
        'material': 1,
        'add': 2,
        'drop': 3,
        'move': 4,
        'no_change': 5
    }

    def __init__(self, split):

        print('Speaker Dataset loading vocab json file: ', vocab_json)
        self.vocab_json = vocab_json
        self.word_to_idx = json.load(open(self.vocab_json, 'r'))
        self.idx_to_word = {}
        for word, idx in self.word_to_idx.items():
            self.idx_to_word[idx] = word
        self.vocab_size = len(self.idx_to_word)
        print('vocab size is ', self.vocab_size)

        self.type_mapping = json.load(open(type_mapping_json, 'r'))
        self.type_to_img = {}
        for k, v in self.type_mapping.items():
            self.type_to_img[k] = set([int(x.split('.')[0]) for x in v])


        self.d_feat_dir = default_feature_dir
        self.s_feat_dir = semantic_feature_dir
        self.n_feat_dir = nonsemantic_feature_dir

        self.d_feats = sorted(os.listdir(self.d_feat_dir))
        self.s_feats = sorted(os.listdir(self.s_feat_dir))
        self.n_feats = sorted(os.listdir(self.n_feat_dir))

        assert len(self.d_feats) == len(self.s_feats) == len(self.n_feats), \
            'The number of features are different from each other!'
        
        self.d_img_dir = default_img_dir
        self.s_img_dir = semantic_img_dir
        self.n_img_dir = nonsemantic_img_dir

        self.d_imgs = sorted(os.listdir(self.d_img_dir))
        self.s_imgs = sorted(os.listdir(self.s_img_dir))
        self.n_imgs = sorted(os.listdir(self.n_img_dir))

        self.splits = json.load(open(splits_json, 'r'))
        self.split = split

        if split == 'train':
            self.batch_size = train_batch_size
            self.seq_per_img = 1
            self.split_idxs = self.splits['train']
            self.num_samples = len(self.split_idxs)
            if max_samples is not None:
                self.num_samples = min(max_samples, self.num_samples)
        elif split == 'val': 
            self.batch_size = val_batch_size
            self.seq_per_img = 5
            self.split_idxs = self.splits['val']
            self.num_samples = len(self.split_idxs)
            if max_samples is not None:
                self.num_samples = min(max_samples, self.num_samples)
        elif split == 'test': 
            self.batch_size = test_batch_size
            self.seq_per_img = 5
            self.split_idxs = self.splits['test']
            self.num_samples = len(self.split_idxs)
            if max_samples is not None:
                self.num_samples = min(max_samples, self.num_samples)
        else:
            raise Exception('Unknown data split %s' % split)

        print("Dataset size for %s: %d" % (split, self.num_samples))

        # load in the sequence data
        self.h5_label_file = h5py.File(h5_label_file, 'r')
        seq_size = self.h5_label_file['labels'].shape
        self.labels = self.h5_label_file['labels'][:] # just gonna load...
        self.neg_labels = self.h5_label_file['neg_labels'][:]
        self.max_seq_length = seq_size[1]
        self.label_start_idx = self.h5_label_file['label_start_idx'][:]
        self.label_end_idx = self.h5_label_file['label_end_idx'][:]
        self.neg_label_start_idx = self.h5_label_file['neg_label_start_idx'][:]
        self.neg_label_end_idx = self.h5_label_file['neg_label_end_idx'][:]
        print('Max sequence length is %d' % self.max_seq_length)
        self.h5_label_file.close()

    def __len__(self):
        return self.num_samples

    def __getitem__(self, index):
        random.seed()
        img_idx = self.split_idxs[index]

        # Fetch image data
        # one easy way to augment data is to use nonsemantically changed
        # scene as the default :)
        if self.split == 'train':
            if random.random() < 0.5:
                d_feat_path = os.path.join(self.d_feat_dir, self.d_feats[img_idx])
                d_img_path = os.path.join(self.d_img_dir, self.d_imgs[img_idx])
                n_feat_path = os.path.join(self.n_feat_dir, self.n_feats[img_idx])
                n_img_path = os.path.join(self.n_img_dir, self.n_imgs[img_idx])
            else:
                d_feat_path = os.path.join(self.n_feat_dir, self.n_feats[img_idx])
                d_img_path = os.path.join(self.n_img_dir, self.n_imgs[img_idx])
                n_feat_path = os.path.join(self.d_feat_dir, self.d_feats[img_idx])
                n_img_path = os.path.join(self.d_img_dir, self.d_imgs[img_idx])
        else:
            d_feat_path = os.path.join(self.d_feat_dir, self.d_feats[img_idx])
            d_img_path = os.path.join(self.d_img_dir, self.d_imgs[img_idx])
            n_feat_path = os.path.join(self.n_feat_dir, self.n_feats[img_idx])
            n_img_path = os.path.join(self.n_img_dir, self.n_imgs[img_idx])


        q_feat_path = os.path.join(self.s_feat_dir, self.s_feats[img_idx])
        q_img_path = os.path.join(self.s_img_dir, self.s_imgs[img_idx])

        d_feature = torch.FloatTensor(np.load(d_feat_path))
        n_feature = torch.FloatTensor(np.load(n_feat_path))
        q_feature = torch.FloatTensor(np.load(q_feat_path))

        # Fetch change type labels
        aux_label_pos = -1
        for type, img_set in self.type_to_img.items():
            if img_idx in img_set:
                aux_label_pos = self.type_to_label[type]
                break
        aux_label_neg = self.type_to_label['no_change']

        # Fetch sequence labels
        ix1 = self.label_start_idx[img_idx]
        ix2 = self.label_end_idx[img_idx]
        n_cap = ix2 - ix1 + 1

        seq = np.zeros([self.seq_per_img, self.max_seq_length + 1], dtype=int)
        if n_cap < self.seq_per_img:
            # we need to subsample (with replacement)
            for q in range(self.seq_per_img):
                ixl = random.randint(ix1, ix2)
                seq[q, :self.max_seq_length] = \
                    self.labels[ixl, :self.max_seq_length]
        else:
            ixl = random.randint(ix1, ix2 - self.seq_per_img + 1)
            seq[:, :self.max_seq_length] = \
                self.labels[ixl: ixl + self.seq_per_img, :self.max_seq_length]

        # Fetch negative sequence labels
        ix1 = self.neg_label_start_idx[img_idx]
        ix2 = self.neg_label_end_idx[img_idx]
        n_cap = ix2 - ix1 + 1

        neg_seq = np.zeros([self.seq_per_img, self.max_seq_length + 1], dtype=int)
        if n_cap < self.seq_per_img:
            # we need to subsample (with replacement)
            for q in range(self.seq_per_img):
                ixl = random.randint(ix1, ix2)
                neg_seq[q, :self.max_seq_length] = \
                    self.neg_labels[ixl, :self.max_seq_length]
        else:
            ixl = random.randint(ix1, ix2 - self.seq_per_img + 1)
            neg_seq[:, :self.max_seq_length] = \
                self.neg_labels[ixl: ixl + self.seq_per_img, :self.max_seq_length]

        # Generate masks
        mask = np.zeros_like(seq)
        nonzeros = np.array(list(map(lambda x: (x != 0).sum() + 1, seq)))
        for ix, row in enumerate(mask):
            row[:nonzeros[ix]] = 1

        neg_mask = np.zeros_like(neg_seq)
        nonzeros = np.array(list(map(lambda x: (x != 0).sum() + 1, neg_seq)))
        for ix, row in enumerate(neg_mask):
            row[:nonzeros[ix]] = 1

        return (d_feature, n_feature, q_feature,
                seq, neg_seq, mask, neg_mask, aux_label_pos, aux_label_neg,
                d_img_path, n_img_path, q_img_path)

    def get_vocab_size(self):
        return self.vocab_size

    def get_idx_to_word(self):
        return self.idx_to_word

    def get_word_to_idx(self):
        return self.word_to_idx

    def get_max_seq_length(self):
        return self.max_seq_length

def rcc_collate(batch):
    transposed = list(zip(*batch))
    d_feat_batch = transposed[0]
    n_feat_batch = transposed[1]
    q_feat_batch = transposed[2]
    seq_batch = default_collate(transposed[3])
    neg_seq_batch = default_collate(transposed[4])
    mask_batch = default_collate(transposed[5])
    neg_mask_batch = default_collate(transposed[6])
    aux_label_pos_batch = default_collate(transposed[7])
    aux_label_neg_batch = default_collate(transposed[8])
    if any(f is not None for f in d_feat_batch):
        d_feat_batch = default_collate(d_feat_batch)
    if any(f is not None for f in n_feat_batch):
        n_feat_batch = default_collate(n_feat_batch)
    if any(f is not None for f in q_feat_batch):
        q_feat_batch = default_collate(q_feat_batch)

    d_img_batch = transposed[9]
    n_img_batch = transposed[10]
    q_img_batch = transposed[11]
    return (d_feat_batch, n_feat_batch, q_feat_batch,
            seq_batch, neg_seq_batch,
            mask_batch, neg_mask_batch,
            aux_label_pos_batch, aux_label_neg_batch,
            d_img_batch, n_img_batch, q_img_batch)

class RCCDataLoader(DataLoader):
    
    def __init__(self, dataset, **kwargs):
        kwargs['collate_fn'] = rcc_collate
        super().__init__(dataset, **kwargs)


##DataLoader

In [ ]:
def create_dataset2(split='train'):

  dataset = RCCDataset(split)
  data_loader = RCCDataLoader(
    dataset,
    batch_size=dataset.batch_size,
    shuffle=True if split == 'train' else False,
    num_workers=8)

    
  return dataset, data_loader

In [ ]:
# Data loading part
#train_dataset, train_loader = create_dataset2('train')
#idx_to_word = train_dataset.get_idx_to_word()
test_dataset, test_loader = create_dataset2('test')

Speaker Dataset loading vocab json file:  /content/cripp_trial/jsons/vocab.json
vocab size is  76
Dataset size for test: 62
Max sequence length is 23


In [ ]:
idx_to_word = test_dataset.get_idx_to_word()

##EVAL LOOP

In [ ]:
set_mode('eval', [change_detector, speaker])
with torch.no_grad():
    test_iter_start_time = time.time()

    result_sents_pos = {}
    result_sents_neg = {}
    for i, batch in tqdm(enumerate(test_loader)):

        d_feats, nsc_feats, sc_feats, \
        labels, no_chg_labels, masks, no_chg_masks, aux_labels_pos, aux_labels_neg, \
        d_img_paths, nsc_img_paths, sc_img_paths = batch

        batch_size = d_feats.size(0)

        d_feats, nsc_feats, sc_feats = d_feats.to(device), nsc_feats.to(device), sc_feats.to(device)
        d_feats, nsc_feats, sc_feats = \
            spatial_info(d_feats), spatial_info(nsc_feats), spatial_info(sc_feats)
        labels, masks = labels.to(device), masks.to(device)
        no_chg_labels, no_chg_masks = no_chg_labels.to(device), no_chg_masks.to(device)
        aux_labels_pos, aux_labels_neg = aux_labels_pos.to(device), aux_labels_neg.to(device)

        chg_pos_logits, chg_pos_att_bef, chg_pos_att_aft, \
        chg_pos_feat_bef, chg_pos_feat_aft, chg_pos_feat_diff = change_detector(d_feats, sc_feats)
        chg_neg_logits, chg_neg_att_bef, chg_neg_att_aft, \
        chg_neg_feat_bef, chg_neg_feat_aft, chg_neg_feat_diff = change_detector(d_feats, nsc_feats)


        speaker_output_pos, _ = speaker._sample(chg_pos_feat_bef,
                                                chg_pos_feat_aft,
                                                chg_pos_feat_diff,
                                                labels, sample_max=1)

        pos_dynamic_atts = speaker.get_module_weights().detach().cpu().numpy()  # (batch, seq_len, 3)

        speaker_output_neg, _ = speaker._sample(chg_neg_feat_bef,
                                                chg_neg_feat_aft,
                                                chg_neg_feat_diff,
                                                no_chg_labels, sample_max=1)

        neg_dynamic_atts = speaker.get_module_weights().detach().cpu().numpy()  # (batch, seq_len, 3)

        gen_sents_pos = decode_sequence(idx_to_word, speaker_output_pos)
        gen_sents_neg = decode_sequence(idx_to_word, speaker_output_neg)

        chg_pos_att_bef = chg_pos_att_bef.cpu().numpy()
        chg_pos_att_aft = chg_pos_att_aft.cpu().numpy()
        chg_neg_att_bef = chg_neg_att_bef.cpu().numpy()
        chg_neg_att_aft = chg_neg_att_aft.cpu().numpy()
        dummy = np.ones_like(chg_pos_att_bef)

        for j in range(batch_size):
            gts = decode_sequence(idx_to_word, labels[j][:, 1:])
            gts_neg = decode_sequence(idx_to_word, no_chg_labels[j][:, 1:])
            sent_pos = gen_sents_pos[j]
            sent_neg = gen_sents_neg[j]
            image_id = d_img_paths[j].split('_')[-1]
            result_sents_pos[image_id] = sent_pos
            result_sents_neg[image_id + '_n'] = sent_neg
            image_num = image_id.split('.')[0]
            att_bef_path = os.path.join(att_output_path, image_num + '_before')
            att_aft_path = os.path.join(att_output_path, image_num + '_after')
            np.save(att_bef_path, chg_pos_att_bef[j])
            np.save(att_aft_path, chg_pos_att_aft[j])


    test_iter_end_time = time.time() - test_iter_start_time
    print('Test took %.4f seconds' % test_iter_end_time)

    result_save_path_pos = os.path.join(caption_output_path, 'sc_results.json')
    result_save_path_neg = os.path.join(caption_output_path, 'nsc_results.json')
    coco_gen_format_save(result_sents_pos, result_save_path_pos)
    coco_gen_format_save(result_sents_neg, result_save_path_neg)


62it [00:16,  3.74it/s]

Test took 16.8172 seconds
